In [ ]:
from IPython.core.display import HTML
with open('./style.css') as f:
    css = f.read()
HTML(css)

# Play Chess Endgames Using Custom Endgame Tablebase

## What Is This Jupyter Notebook About?

The following notebook extends the class` ChessEndgame`, so that beside the Gaviota tablebases also custom tablebases can be used. For this purpose the class `CustomTablebase` is implemented first. This replaces the `chess.Gaviota.NativeTablebase` and also allows to query the depth-to-mate of a given board. It is based on the sets calculated by the Retrograde Analysis. In addition, the `ChessEndgame` class method `open_tablebase` is overwritten / extended so that the `CustomTablebase` class can be returned instead of the `chess.Gaviota.NativeTablebase`.

## Imports And Preperations

In [ ]:
import chess
import chess.gaviota
import nbimporter
from RetrogradeAnalysis import board_to_int
from typing import Union, List, Set, Dict

In [ ]:
%%capture
%run ChessEndgame.ipynb

## Implementation Of The CustomTablebase Class

First, we need the CustomTablebase class. This only has to implement the method `probe_dtm`. The idea now is that we load in our own custom tablebase instead of the Gaviota tablebase and use it to determine the depth-to-mate. 

To do this, first open the corresponding file in the `__init__` method that is called during initialization (based on the pieces_str). As described in the previous notebook, such a file contains the sets of the respective positions as a serialized binary. It can be deserialized via `pickle.load`. So the first set in the list contains all checkmate positions, the second set in the list contains all depth-to-mate 1 position, etc.

Next comes the actual implementation of the `probe_dtm` method. This first converts the current board into the corresponding integer representation, as these are stored within the sets. Then we enumerate through the list of sets and form the intersection with the set and the set, which contains only the integer representation. In this way, we can very quickly check whether the board is present in the corresponding set. If not, the following set is simply checked, if so, we have a hit and based on the "set numbering" already the absolute value of depth-to-mate. At this point, all we have to do is check whether it is even or odd. Even means that the depth-to-mate is returned as a negative value since the player who is checkmated on the turn in an even number of moves. Analogously, an odd depth-to-mate means that the player on the move can force Checkmate. So depth-to-mate is returned as a positive value

In addition, we declare the `close` method. This is not necessarily required for the CustomTablebase. because no file remains open, but it is called in the `play_game` method, so it should be declared.

In [1]:
class CustomTablebase():
    
    def __init__(self, directory: str, piece_str: str):
        self.piece_str = piece_str
        with open(f'{directory}/{piece_str}.txt', 'r') as tablebase_file:
            self._sets = pickle.load(tablebase_file)
    
    def probe_dtm(self, board):
        board_id = board_to_int(board, self.piece_str)
        for dtm, s in enumerate(self._sets):
            if s & {board_id}: 
                if dtm % 2 == 0: return -dtm
                else:            return dtm
        return 0 
    
    def close(self):
        pass

## Extension Of The `open_tablebase` Method

**`def open_tablebase(self, tablebase_config: Dict[str, Union[str, bool]]) -> Union[chess.gaviota.NativeTablebase, CustomTablebase]`**

Next, the `open_tablebase` method of the `ChessEndgame` class must now be overridden so that a `tablebase_config` with the parameter `custom = True` returns an instance of the previously implemented `CustomTablebase` class. For this a helper method `board_to_piece_str` is needed, because the piece_str must be given to the constructor of the `CustomTablebase`, so that the correct tablebase can be opened and loaded. This method gets the values of the `piece_map`, converts the `chess.Piece`s then into its appropriate symbols (e.g. `white king = 'K'`) and creates the needed string with a custom sorting order (`K > Q > R > N > B > P > k > q > r > n > b > p`).

In [ ]:
def open_tablebase(self, tablebase_config: Dict[str, Union[str, bool]], board: chess.Board) -> Union[chess.gaviota.NativeTablebase, CustomTablebase]:
    if tablebase_config['custom']:
        piece_str = self._board_to_piece_str(board)
        return CustomTablebase(tablebase_config['directory'], piece_str)
    else:
        return chess.gaviota.open_tablebase(tablebase_config['directory'])
    
setattr(ChessEndgame, '_open_tablebase', open_tablebase)
del open_tablebase

In [ ]:
def board_to_piece_str(self, board: chess.Board) -> str:
    SORT_ORDER = dict(zip(['K','Q','R','N','B','P','k','q','r','n','b','p'], [0,1,2,3,4,5,6,7,8,9,10,11]))
    piece_list = list(piece.symbol() for piece in board.piece_map().values())
    piece_list.sort(key=lambda symbol: SORT_ORDER[symbol])
    return ''.join(piece_list)
    
setattr(ChessEndgame, '_board_to_piece_str', board_to_piece_str)
del board_to_piece_str

## Examples Of Playing With The Extended ChessEndgame Class (Support Gaviota and Custom Tablebases)

### Playing With `EPD/FEN-String` (King, Rook vs. King, Queen)

In [ ]:
ChessEndgame().play_game('8/8/8/7q/8/5k2/K7/2R5 w - -', {'directory': './tables', 'custom': True}, False)

### Playing With `piece_str` (King vs. King, Bishop, Knight)

In [ ]:
ChessEndgame().play_game('Kkbn', {'directory': './tables', 'custom': True}, False)

### Playing With `piece_list` (King vs. King, Rook)

In [ ]:
piece_list = [chess.Piece(chess.KING, chess.WHITE), chess.Piece(chess.KING, chess.BLACK), chess.Piece(chess.ROOK, chess.BLACK)]
ChessEndgame().play_game(piece_list, {'directory': './tables', 'custom': True}, False)

### Playing With `winning_color` (Random Endgame, Black Can Force Win)

In [ ]:
ChessEndgame().play_game(chess.BLACK, {'directory': './tables', 'custom': True}, False)

### Playing With `winning_color` (Random Endgame, White Can Force Win)

In [ ]:
ChessEndgame().play_game(chess.WHITE, {'directory': './tables', 'custom': True}, False)

### Playing With `EPD/FEN-String` And `export_only` (King, Rook vs. King, Queen)

In [ ]:
ChessEndgame().play_game('8/8/8/7q/8/5k2/K7/2R5 w - -', {'directory': './tables', 'custom': True}, True)